In [ ]:
endpoint = "https://kdb17ocr.cognitiveservices.azure.com/"
key = "ERNdq31sYC7BGzVv1gn3BPE24gGzOpueHElkMGRCY03OGcpcLrxiJQQJ99BDACYeBjFXJ3w3AAALACOGidId"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install azure-ai-formrecognizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://kdb17ocr.cognitiveservices.azure.com/"
key = "ERNdq31sYC7BGzVv1gn3BPE24gGzOpueHElkMGRCY03OGcpcLrxiJQQJ99BDACYeBjFXJ3w3AAALACOGidId"

client = DocumentAnalysisClient(endpoint, AzureKeyCredential(key))

In [ ]:
import os
import time
from PIL import Image
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Replace with your Azure endpoint and key

endpoint = "https://kdb17ocr.cognitiveservices.azure.com/"
key = "ERNdq31sYC7BGzVv1gn3BPE24gGzOpueHElkMGRCY03OGcpcLrxiJQQJ99BDACYeBjFXJ3w3AAALACOGidId"

client = DocumentAnalysisClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(key)
)

input_folder = "/content/drive/MyDrive/2000_deeds"
output_folder = "/content/drive/MyDrive/ocr_output_read"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def wait_for_file(filepath, timeout=10):
    start = time.time()
    while not os.path.exists(filepath):
        if time.time() - start > timeout:
            return False
        time.sleep(0.5)  # Check every 0.5 sec
    return True

tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]

for tif_filename in tif_files:
    tif_path = os.path.join(input_folder, tif_filename)
    combined_text = ""

    if not wait_for_file(tif_path):
        print(f"❌ File not found after waiting: {tif_path} — skipping")
        continue

    try:
        im = Image.open(tif_path)
        num_pages = getattr(im, "n_frames", 1)
        print(f"📎 {tif_filename} has {num_pages} pages")

        for page_index in range(num_pages):
            im.seek(page_index)
            temp_page_path = f"/tmp/temp_page_{page_index}.tif"
            im.save(temp_page_path)

            with open(temp_page_path, "rb") as f:
                poller = client.begin_analyze_document("prebuilt-read", f)
                result = poller.result()

            combined_text += f"\n\n========== Page {page_index + 1} ==========\n\n"

            for page in result.pages:
                for line in page.lines:
                    line_text = line.content.strip()
                    word_count = len(line_text.split())

                    if word_count > 10:
                        combined_text += line_text + "\n"
                        combined_text += "<laysep@@##$$>\n\n"
                    else:
                        combined_text += line_text + " "

            combined_text += "\n"

        output_text_filename = os.path.join(
            output_folder, f"{os.path.splitext(tif_filename)[0]}.txt"
        )

        with open(output_text_filename, "w", encoding="utf-8") as f:
            f.write(combined_text)

        print(f"✅ Processed {tif_filename}: Text saved to {output_text_filename}")

    except Exception as e:
        print(f"❌ Failed to process {tif_filename}: {str(e)}")

print("🎉 All TIF files processed. Text saved in Google Drive.")


📎 2025041001048943-0.tif has 4 pages
✅ Processed 2025041001048943-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001048943-0.txt
📎 2025041001049396-0.tif has 4 pages
✅ Processed 2025041001049396-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001049396-0.txt
📎 2025041001048826-0.tif has 4 pages
✅ Processed 2025041001048826-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001048826-0.txt
📎 2025041001048778-0.tif has 6 pages
✅ Processed 2025041001048778-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001048778-0.txt
📎 2025041001048777-0.tif has 16 pages
✅ Processed 2025041001048777-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001048777-0.txt
📎 2025041001049430-0.tif has 6 pages
✅ Processed 2025041001049430-0.tif: Text saved to /content/drive/MyDrive/ocr_output_read/2025041001049430-0.txt
📎 2025041001049406-0.tif has 6 pages
✅ Processed 2025041001049406-0.tif: Text saved to /content/drive/MyD